## Overview ##

Based on Starter Kernel [Logistic Regression on Tournament Seeds by Kasper P. Lauritzen](https://www.kaggle.com/kplauritzen/notebookde27b18258?scriptVersionId=804590) 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from subprocess import check_output
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix



/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load the training data ##
We're keeping it simple & using only 2 files for this model: the Tourney seeds & Compact results.

In [133]:
data_dir = '../input/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
#df_tour = pd.read_csv(data_dir + 'NCAATourneyDetailedResults.csv')
df_tour = pd.read_csv(data_dir + 'RegularSeasonCompactResults.csv')
teams = pd.read_csv(data_dir + 'Teams.csv')

def seed_to_int(seed):
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()


,Season,WTeamID,LTeamID
0,1985,1228,1328
1,1985,1106,1354
2,1985,1112,1223
3,1985,1165,1432
4,1985,1192,1447


Build out the id string (year_wteam_lteam)
Create a prediction at the end.

In [159]:
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_true=df_concat.copy()

columnsTitles=["LTeamID","WTeamID"]
df_false = df_concat.rename(columns={'WTeamID': 'LTeamID_', 'LTeamID': 'WTeamID_'}) #idk if i have to do this I just didnt want to create issues
df_false = df_false.rename(columns={'LTeamID_': 'LTeamID', 'WTeamID_': 'WTeamID'})
df_false = df_false.rename(columns={'WSeed': 'LSeed_', 'LSeed': 'WSeed_'}) #idk if i have to do this I just didnt want to create issues
df_false = df_false.rename(columns={'LSeed_': 'LSeed', 'WSeed_': 'WSeed'})


df_true['label'] = 1
df_false['label'] = 0

df = df_true.append(df_false, ignore_index=True)
df = df.fillna(17)

#df['WTID']=df.apply(build_wteamyear, axis=1) #again, probably better way to do this and save a function
#df['LTID']=df.apply(build_lteamyear, axis=1)
df['SeedDiff'] =  df.WSeed - df.LSeed #Lower seeds are better, so Negative means that the lower seed team won.
df = df[df['Season'] > 2013] # We want to train on 2014 and above.

df = df[['WTeamID', 'LTeamID', 'SeedDiff', 'label']]
df.head()



,WTeamID,LTeamID,SeedDiff,label
15289,1103,1157,1.0,1
15290,1279,1157,1.0,1
15291,1278,1157,1.0,1
15292,1418,1157,-2.0,1
15293,1155,1157,1.0,1


## Encode and, Scale and Split ##

In [160]:
X = df.iloc[:, 0:3].values
y = df.iloc[:, 3].values


enc_1 = LabelEncoder()
X[:, 0] = enc_1.fit_transform(X[:, 0])
enc_2 = LabelEncoder()
X[:, 1] = enc_2.fit_transform(X[:, 1])
onehotencoder = OneHotEncoder(categorical_features = [0,1],handle_unknown='error') #handle_unknown='unknown'or impute
X = onehotencoder.fit_transform(X).toarray()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ds_width=X_test.shape[1]


## Build the classifier and hit the button ##

In [164]:
from keras.layers import LeakyReLU
from keras import metrics
from keras import optimizers

# 61%  //Dense(6, he_normal)//leakyrelu alpha 0.2//Dense(16, he_normal)
# leakyrelu alpha 0.2//Dense(16, he_normal)//leakyrelu alpha 0.2//Dense(1, he_normal, sigmoid)//SGD, Mean_Squared_Error
# Third Attempt.
classifier = Sequential()
classifier.add(Dense(6,input_shape=(ds_width,),kernel_initializer = 'normal',activation='relu'))
#classifier.add(LeakyReLU(alpha=0.2))
classifier.add(Dense(13, activation='relu',kernel_initializer = 'normal',))
#classifier.add(LeakyReLU(alpha=0.2))
classifier.add(Dense(13, activation='relu',kernel_initializer = 'normal',))
classifier.add(Dense(1,kernel_initializer = 'normal' ))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['binary_accuracy']) #binary_crossentropy
print("Cooking... Please wait")
classifier.fit(X_train, y_train, batch_size = 250, epochs = 500, verbose=1)
print("*ding*... Testing Accuracy on test set")
#Run Predictions on the test set
pred_test = classifier.predict(X_test)

pred_test = (pred_test > 0.5)
cm = confusion_matrix(pred_test, y_test)
print ((cm[0][0]+cm[1][1])/(sum(cm[0])+sum(cm[1])))

Cooking... Please wait
Epoch 1/500
3891/3891 [==============================] - 4s 964us/step - loss: 2.5723 - binary_accuracy: 0.5009
Epoch 2/500
3891/3891 [==============================] - 0s 20us/step - loss: 1.8436 - binary_accuracy: 0.5009
Epoch 3/500
3891/3891 [==============================] - 0s 23us/step - loss: 1.5649 - binary_accuracy: 0.5009
Epoch 4/500
3891/3891 [==============================] - 0s 20us/step - loss: 1.2945 - binary_accuracy: 0.5009
Epoch 5/500
3891/3891 [==============================] - 0s 26us/step - loss: 1.0318 - binary_accuracy: 0.5012
Epoch 6/500
3891/3891 [==============================] - 0s 26us/step - loss: 0.8059 - binary_accuracy: 0.5307
Epoch 7/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.6410 - binary_accuracy: 0.6076
Epoch 8/500
3891/3891 [==============================] - 0s 24us/step - loss: 0.5550 - binary_accuracy: 0.6831
Epoch 9/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.5200 - b

3891/3891 [==============================] - 0s 25us/step - loss: 0.2924 - binary_accuracy: 0.6641
Epoch 75/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2917 - binary_accuracy: 0.6659
Epoch 76/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2905 - binary_accuracy: 0.6667
Epoch 77/500
3891/3891 [==============================] - 0s 25us/step - loss: 0.2897 - binary_accuracy: 0.6654
Epoch 78/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2891 - binary_accuracy: 0.6667
Epoch 79/500
3891/3891 [==============================] - 0s 23us/step - loss: 0.2883 - binary_accuracy: 0.6662
Epoch 80/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.2876 - binary_accuracy: 0.6641
Epoch 81/500
3891/3891 [==============================] - 0s 29us/step - loss: 0.2879 - binary_accuracy: 0.6605
Epoch 82/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2866 - binary_accuracy: 0.6651
Epoch

3891/3891 [==============================] - 0s 26us/step - loss: 0.2631 - binary_accuracy: 0.6530
Epoch 147/500
3891/3891 [==============================] - 0s 28us/step - loss: 0.2629 - binary_accuracy: 0.6530
Epoch 148/500
3891/3891 [==============================] - 0s 28us/step - loss: 0.2618 - binary_accuracy: 0.6510
Epoch 149/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2622 - binary_accuracy: 0.6530
Epoch 150/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.2612 - binary_accuracy: 0.6543
Epoch 151/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2614 - binary_accuracy: 0.6551
Epoch 152/500
3891/3891 [==============================] - 0s 17us/step - loss: 0.2611 - binary_accuracy: 0.6538
Epoch 153/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.2619 - binary_accuracy: 0.6500
Epoch 154/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2608 - binary_accuracy: 0.65

3891/3891 [==============================] - 0s 21us/step - loss: 0.2676 - binary_accuracy: 0.6415
Epoch 219/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2676 - binary_accuracy: 0.6389
Epoch 220/500
3891/3891 [==============================] - 0s 18us/step - loss: 0.2650 - binary_accuracy: 0.6438
Epoch 221/500
3891/3891 [==============================] - 0s 26us/step - loss: 0.2662 - binary_accuracy: 0.6428
Epoch 222/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2665 - binary_accuracy: 0.6389
Epoch 223/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2662 - binary_accuracy: 0.6435
Epoch 224/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.2660 - binary_accuracy: 0.6420
Epoch 225/500
3891/3891 [==============================] - 0s 28us/step - loss: 0.2658 - binary_accuracy: 0.6399
Epoch 226/500
3891/3891 [==============================] - 0s 28us/step - loss: 0.2667 - binary_accuracy: 0.63

3891/3891 [==============================] - 0s 21us/step - loss: 0.2574 - binary_accuracy: 0.6651
Epoch 291/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2571 - binary_accuracy: 0.6646
Epoch 292/500
3891/3891 [==============================] - 0s 26us/step - loss: 0.2568 - binary_accuracy: 0.6638
Epoch 293/500
3891/3891 [==============================] - 0s 18us/step - loss: 0.2565 - binary_accuracy: 0.6638
Epoch 294/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2565 - binary_accuracy: 0.6636
Epoch 295/500
3891/3891 [==============================] - 0s 23us/step - loss: 0.2561 - binary_accuracy: 0.6654
Epoch 296/500
3891/3891 [==============================] - 0s 28us/step - loss: 0.2558 - binary_accuracy: 0.6631
Epoch 297/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2556 - binary_accuracy: 0.6605
Epoch 298/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2556 - binary_accuracy: 0.66

3891/3891 [==============================] - 0s 18us/step - loss: 0.2475 - binary_accuracy: 0.6497
Epoch 363/500
3891/3891 [==============================] - 0s 17us/step - loss: 0.2476 - binary_accuracy: 0.6500
Epoch 364/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2471 - binary_accuracy: 0.6497
Epoch 365/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2470 - binary_accuracy: 0.6525
Epoch 366/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2470 - binary_accuracy: 0.6507
Epoch 367/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2463 - binary_accuracy: 0.6515
Epoch 368/500
3891/3891 [==============================] - 0s 27us/step - loss: 0.2466 - binary_accuracy: 0.6484
Epoch 369/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2468 - binary_accuracy: 0.6464
Epoch 370/500
3891/3891 [==============================] - 0s 21us/step - loss: 0.2467 - binary_accuracy: 0.65

3891/3891 [==============================] - 0s 22us/step - loss: 0.2410 - binary_accuracy: 0.6430
Epoch 435/500
3891/3891 [==============================] - 0s 23us/step - loss: 0.2409 - binary_accuracy: 0.6423
Epoch 436/500
3891/3891 [==============================] - 0s 23us/step - loss: 0.2412 - binary_accuracy: 0.6423
Epoch 437/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2408 - binary_accuracy: 0.6410
Epoch 438/500
3891/3891 [==============================] - 0s 26us/step - loss: 0.2409 - binary_accuracy: 0.6381
Epoch 439/500
3891/3891 [==============================] - 0s 20us/step - loss: 0.2408 - binary_accuracy: 0.6361
Epoch 440/500
3891/3891 [==============================] - 0s 17us/step - loss: 0.2413 - binary_accuracy: 0.6402
Epoch 441/500
3891/3891 [==============================] - 0s 19us/step - loss: 0.2408 - binary_accuracy: 0.6412
Epoch 442/500
3891/3891 [==============================] - 0s 22us/step - loss: 0.2411 - binary_accuracy: 0.63

## Make the prediction ##

In [166]:
def format_dataset(df_sample):
    df_pred = pd.DataFrame()
    df_pred['Season'], df_pred['WTeamID'], df_pred['LTeamID'] = df_sample_sub['ID'].str.split('_', 2).str
    df_pred = df_pred.astype(int) #DO I NEED THIS?
    df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
    df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
    df_preddummy = pd.merge(left=df_pred, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
    df_predmerge = pd.merge(left=df_preddummy, right=df_lossseeds, on=['Season', 'LTeamID'])
    df_predmerge = df_predmerge.fillna(17)
    df_predmerge['SeedDiff'] = df_predmerge.WSeed - df_predmerge.LSeed
    df_predmerge = df_predmerge.rename(columns={'Season':'Year'})
    df_predictions = df_predmerge[['WTeamID','LTeamID','SeedDiff']] # We really should add year to this somewhere
    df_predictions['WTeamID'] = df_predictions['WTeamID'].astype(str)
    df_predictions['LTeamID'] = df_predictions['LTeamID'].astype(str)
    for x in df_predictions.columns:
        df_predictions[x]=df_predictions[x].astype(df_preenc[x].dtypes.name)
    return df_predictions.iloc[:, 0:3].values



#Convert the prediction table to our format
df_predX = format_dataset(pd.read_csv(data_dir + 'SampleSubmissionStage1.csv'))
#Encode the Predictions
df_predX[:, 0] = enc_1.transform(df_predX[:, 0])
df_predX[:, 1] = enc_2.transform(df_predX[:, 1])
df_predX = onehotencoder.transform(df_predX).toarray()
df_predX = sc.transform(df_predX)
#Generate the predictions and save
y_p = classifier.predict(df_predX)
clipped_preds = np.clip(y_p, 0.0001, 0.999)
df_out=df_sample_sub
df_out.Pred = clipped_preds
df_out.to_csv('out.csv', index=False)
display(df_out)
print(df_out['Pred'].std())


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

,ID,Pred
0,2014_1107_1110,0.005108
1,2014_1107_1112,0.005108
2,2014_1107_1113,0.005108
3,2014_1107_1124,0.005108
4,2014_1107_1140,0.005108
5,2014_1107_1142,0.454111
6,2014_1107_1153,0.005108
7,2014_1107_1157,0.005108
8,2014_1107_1160,0.999000
9,2014_1107_1163,0.000100


0.4399929344654083
